# PDF AI
The objective of this is to test how to embedd data from pdf

In [5]:
import os
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import chromadb
os.environ["LANGSMITH_PROJECT"] = "data-chunking-pdf"
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
chroma_client = chromadb.HttpClient(host='localhost', port=8000)

data = "user-manual"
collection_name = "user-manual"

vector_store = Chroma(
    collection_name=collection_name,
    embedding_function=embeddings,
    client=chroma_client
)
file_path = (
    f'../data/{data}'
)
files_pdf = []
for file in os.listdir(file_path):
    if file.endswith(".pdf"):
        files_pdf.append(os.path.join(file_path, file))

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
for file in files_pdf:
    loader = PyPDFLoader(file)
    document = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunked_documents = text_splitter.split_documents(document)
    vector_store.add_documents(chunked_documents)

    print(f"Added {len(chunked_documents)} chunks to chroma db")

Added 1311 chunks to chroma db


In [7]:
vector_store.similarity_search("rag")

[Document(id='e9e37f89-9644-4f2d-aa59-5259b1e758a4', metadata={'creationdate': '2021-04-14T12:50:24+00:00', 'creator': 'AH XSL Formatter V6.6 MR11 for Windows (x64) : 6.6.13.42545 (2020-02-03T11:23+09)', 'moddate': '2021-06-22T12:41:11+00:00', 'page': 383, 'page_label': '384', 'producer': 'Antenna House PDF Output Library 6.6.1548 (Windows (x64))', 'source': '../data/user-manual/CUPRA_FORMENTOR_06_21_EN.pdf', 'title': "CUPRA Formentor Owner's Manual", 'total_pages': 391, 'trapped': '/false'}, page_content='frequency bands . . . . . . . . . . . . . . . . . . . . . . . . 187\npreset buttons . . . . . . . . . . . . . . . . . . . . . . . . . . 187\nsymbols . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 187\nTraffic information (TP) . . . . . . . . . . . . . . . . . 188\ntune station . . . . . . . . . . . . . . . . . . . . . . . . . . . . 187\nRadio-operated remote control\nsee Keys . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 92\nRadio mode . . . . . . .